## Série histórica IBOVESPA

Análise preditiva da IBOVESPA

**Datasets:** http://www.bmfbovespa.com.br/pt_br/servicos/market-data/historico/mercado-a-vista/series-historicas/

**By:** Gabriel Machado Lopes Gonçalves (GMLG)

In [1]:
# Packages
from pyspark.sql.functions import substring
from pyspark.sql import Row
import pandas as pd

In [2]:
print(sc)

<SparkContext master=local[*] appName=PySparkShell>


In [3]:
# read txt
ibov_v1 = sc.textFile('datasets/COTAHIST_A2018.TXT')

In [4]:
# Clean header line
header = ibov_v1.first()
ibov_v2 = ibov_v1.filter(lambda line: line != header)

In [5]:
# Split lines
ibov_v3 = ibov_v2.map(lambda x: x.split("\t"))

In [6]:
# Convert to dataframe 
ibov_v4 = spark.createDataFrame(ibov_v3)

In [7]:
ibov_v5 = ibov_v4.select(substring(ibov_v4[0], 1, 2).alias('TIPREG'), 
                         substring(ibov_v4[0], 3, 8).alias('DT_PREGAO'), 
                         substring(ibov_v4[0], 11, 2).alias('CODBDI'), 
                         substring(ibov_v4[0], 13, 12).alias('CODNEG'), 
                         substring(ibov_v4[0], 25, 27).alias('TPMERC'), 
                         substring(ibov_v4[0], 28, 12).alias('NOMRES'), 
                         substring(ibov_v4[0], 40, 10).alias('ESPECI'), 
                         substring(ibov_v4[0], 53, 4).alias('MODREF'),
                         substring(ibov_v4[0], 57, 13).alias('PREABE'), # Preco de abertura
                         substring(ibov_v4[0], 70, 13).alias('PREMAX'), # Preco maximo
                         substring(ibov_v4[0], 83, 13).alias('PREMIN'), 
                         substring(ibov_v4[0], 96, 13).alias('PREMED'),
                         substring(ibov_v4[0], 109, 13).alias('PREULT'),
                         substring(ibov_v4[0], 122, 13).alias('PREOFC'),
                         substring(ibov_v4[0], 135, 13).alias('PREOFV'), 
                         substring(ibov_v4[0], 148, 5).alias('TOTNEG'), 
                         substring(ibov_v4[0], 153, 18).alias('QUATOT'), 
                         substring(ibov_v4[0], 135, 13).alias('PREOFV')) 

In [ ]:
# Show rows

#for row in ibov_v5 : print(row)

In [8]:
# Schema
ibov_v5.printSchema()

root
 |-- TIPREG: string (nullable = true)
 |-- DT_PREGAO: string (nullable = true)
 |-- CODBDI: string (nullable = true)
 |-- CODNEG: string (nullable = true)
 |-- TPMERC: string (nullable = true)
 |-- NOMRES: string (nullable = true)
 |-- ESPECI: string (nullable = true)
 |-- MODREF: string (nullable = true)
 |-- PREABE: string (nullable = true)
 |-- PREMAX: string (nullable = true)
 |-- PREMIN: string (nullable = true)
 |-- PREMED: string (nullable = true)
 |-- PREULT: string (nullable = true)
 |-- PREOFC: string (nullable = true)
 |-- PREOFV: string (nullable = true)
 |-- TOTNEG: string (nullable = true)
 |-- QUATOT: string (nullable = true)
 |-- PREOFV: string (nullable = true)



In [9]:
# Save temporary table
ibov_v5.createOrReplaceTempView("ibov_tb")

In [13]:
# Show table
sqlCtx.sql("select * from ibov_tb limit 10").show()

+------+---------+------+------------+--------------------+------------+----------+------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------+------------------+-------------+
|TIPREG|DT_PREGAO|CODBDI|      CODNEG|              TPMERC|      NOMRES|    ESPECI|MODREF|       PREABE|       PREMAX|       PREMIN|       PREMED|       PREULT|       PREOFC|       PREOFV|TOTNEG|            QUATOT|       PREOFV|
+------+---------+------+------------+--------------------+------------+----------+------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------+------------------+-------------+
|    01| 20180102|    02|AALR3       |010ALLIAR      ON...|ALLIAR      |ON      NM|  R$  |0000000001494|0000000001516|0000000001470|0000000001484|0000000001489|0000000001475|0000000001489| 00403|000000000000094500|0000000001489|
|    01| 20180102|    96|AALR3F      |020ALLIAR      ON...|ALLIAR      |ON      NM| 